## Hydrograph ML
Last edited by NC
TAKE 2

Can we predict the hydograph downsteam, using upstream weirs and datasets?

This will be for Golden, CO

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import metpy for meteorological data analysis and visualization
import metpy

# Import dataretrieval for accessing environmental data from various sources
import dataretrieval
import dataretrieval.nwis as nwis

# Import cartopy for cartographic projections and geographic data visualization
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Import USCOUNTIES from metpy.plots for plotting US county boundaries
from metpy.plots import USCOUNTIES

# Import gridliner from cartopy.mpl for formatting gridlines on maps
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


from xgboost import XGBRegressor
from sklearn.utils import resample
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print('What version of Dataretrieval?', dataretrieval.__version__ )

In [3]:
# specify the USGS site code for which we want data.
site = ['06719505', # Clear Creek at Golden
        '06718550', # North Clear Creek @ BlackHawk
        '06716500',
        '06714800', # Leavenworth Creek at Mouth Near Georgetown, CO
        '06716100'] # West Fork Clear Creek Abv Mouth NR Empire, CO

In [4]:
# get instantaneous values (iv)
df = nwis.get_record(sites=site, service='iv', start='2011-1-01', end='2023-12-30')

# Want some metadata
df_site = nwis.get_record(sites=site, service='site')

In [ ]:
df_site

## What are some of these columns?

00060 = Daily Mean Discharge

00065 = Gage height, feet

_cd = condition code

## Plotting a site map

Where are we in the world :)

In [ ]:
# Ensure CRS matches the data
proj = ccrs.LambertConformal(central_longitude=-105.5, central_latitude=39.7)

# Create the figure and axis
fig = plt.figure(figsize=(12, 7))
ax1 = fig.add_subplot(1, 1, 1, projection=proj)

# Set map extent to focus on Florida
ax1.set_extent([-106, -104.5, 39, 40.5], ccrs.Geodetic())  # Adjust the extent to focus on Florida

# Add map features
ax1.add_feature(USCOUNTIES.with_scale('500k'))

# Scatter plot
scatter = ax1.scatter(df_site.dec_long_va, df_site.dec_lat_va, color='red', marker='o', s=100, transform=ccrs.PlateCarree())

# Add legend
ax1.legend([scatter], ['Gauge Locations'], loc='lower left')  # Adjust the legend label as needed

# Add gridlines with tick marks
gl = ax1.gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 12}
gl.ylabel_style = {'size': 12}

plt.show()

## Let's dig into the data

In [ ]:
df.head(3)

I personally hate multi-index, but to each their own

In [ ]:
df = df.reset_index()
df.head()

## Let's do some feature engineering

I am going to break out year, day of year, and hour of the day into it's own column. This is just to make it easier to parse later.

In [ ]:
df['year'] = df['datetime'].dt.year
df['day_of_year'] = df['datetime'].dt.dayofyear
df['hour_of_day'] = df['datetime'].dt.hour

df.dtypes

I will not be using actual datetime anymore

In [10]:
df = df.drop('datetime', axis=1)

In [ ]:
original_shape = df.shape[0]
df = df.drop_duplicates(keep='first')  # Explicitly keep the first occurrence
new_shape = df.shape[0]
print(f"Removed {original_shape - new_shape} duplicate rows")

In [ ]:
df.describe()

## Some EDA

Can make a quick crossplot comparing daily mean discharge to gauge height

In [ ]:
plt.scatter(df['00060'],
            df['00065'], 
            alpha=0.05)

plt.xlabel('Discharge (cfs)')  # Label for x-axis
plt.ylabel('Gauge Height (feet)')  # Label for y-axis
plt.title('USGS Streamflow Data')  # Title for the plot

plt.show()

In [ ]:
# Get unique site numbers from the DataFrame
unique_site_numbers = df['site_no'].unique()

# Get the colormap and generate the required number of colors
cmap = plt.colormaps.get_cmap('tab10')  # Get the colormap
colors = [cmap(i / len(unique_site_numbers)) for i in range(len(unique_site_numbers))]  # Generate the colors

# Create a dictionary to map each site number to a color
site_color_map = {site: colors[i] for i, site in enumerate(unique_site_numbers)}

# Scatter plot with colors based on site numbers
plt.scatter(df['00060'],
            df['00065'],
            c=df['site_no'].map(site_color_map),  # Map each site to its corresponding color
            alpha=0.5,
            s=7)

plt.xlabel('Discharge (cfs)')  # Label for x-axis
plt.ylabel('Gauge Height (feet)')  # Label for y-axis
plt.title('USGS Streamflow Data')  # Title for the plot

plt.xscale('log')  # Applying log scale to the x-axis

# Add a legend
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=site, markerfacecolor=color, markersize=5) for site, color in site_color_map.items()]
plt.legend(handles=legend_handles)

plt.grid(True)  # Adding a grid to the plot

plt.show()

In [ ]:
# Get unique site numbers
unique_site_numbers = df['site_no'].unique()

# Determine the number of rows and columns for subplots
num_rows = len(unique_site_numbers)
num_cols = 1  # One column for each site

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 5*num_rows))

# Plot data for each site
for i, site_number in enumerate(unique_site_numbers):
    # Filter data for the specific site
    site_data = df[df['site_no'] == site_number]
    
    # Plot data on the corresponding subplot
    sns.lineplot(x='day_of_year', y='00060', data=site_data, hue='year', ax=axes[i], alpha=0.5)
    
    # Calculate median discharge for each day of the year
    median_discharge = site_data.groupby('day_of_year')['00060'].median()
    
    # Plot median line on the same subplot
    axes[i].plot(median_discharge.index, median_discharge.values, color='black', linestyle='--', label='Median', linewidth=2)
    
    # Smooth the median discharge using a 7-point convolution
    smoothed_discharge = np.convolve(median_discharge, np.ones(14)/14, mode='same')
    
    # Plot smoothed line on the same subplot
    axes[i].plot(median_discharge.index, smoothed_discharge, color='red', linestyle=':', label='Smoothed', linewidth=2)
    
    # Customize subplot
    axes[i].set_xlabel('Day of Year')
    axes[i].set_ylabel('Discharge (cfs)')
    axes[i].set_title(f'Site {site_number}')
    axes[i].legend(title='Year', loc='upper left')
    axes[i].tick_params(axis='x', rotation=45)  # Rotate x-axis labels for better readability

# Adjust layout
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
nan_counts = df.isna().sum()
nan_counts

In [ ]:
unique_counts = df['year'].value_counts()
unique_counts

In [ ]:
import calendar 

# Assuming df.day_of_year is your data
plt.figure(figsize=(12, 6))
plt.hist(df.day_of_year, bins=52, density=True, color='skyblue', edgecolor='black')

# Improve x-axis labels
months = calendar.month_abbr[1:]
plt.xticks(np.arange(15, 365, 30.5), months, rotation=45)

# Add labels and title
plt.xlabel('Month')
plt.ylabel('Normalized Frequency')
plt.title('Distribution of Events Throughout the Year')

# Add grid
plt.grid(axis='y', alpha=0.75)

# Adjust layout and display
plt.tight_layout()
plt.show()


### Let's see how many nan's per station:

In [ ]:
# in absolute terms
df.isna().sum()

In [ ]:
# As a percentage
(df.isna().sum() / len(df)) * 100

In [21]:
def nans_per_station(df):
    # Get all columns except 'site_no'
    columns_to_check = [col for col in df.columns if col != 'site_no']
    
    # Initialize the result DataFrame with 'site_no'
    result = df['site_no'].drop_duplicates().to_frame()
    
    for column in columns_to_check:
        # Group by 'site_no' and sum NaN values for the current column
        column_nans = df.groupby('site_no')[column].apply(lambda x: x.isna().sum()).reset_index()
        column_nans.columns = ['site_no', f'{column}_nan_count']
        
        # Merge with the result DataFrame
        result = result.merge(column_nans, on='site_no', how='left')
    
    # Add total NaN count per station
    nan_columns = [col for col in result.columns if col.endswith('_nan_count')]
    result['total_nan_count'] = result[nan_columns].sum(axis=1)
    
    # Sort by total NaN count in descending order
    result = result.sort_values('total_nan_count', ascending=False)
    
    return result

In [ ]:
nans_per_station(df)

## We will want to impute the data with the median:

In [23]:
def fill_with_median(df, column):
    # Store the original NaN count
    original_nan_count = df[column].isna().sum()
    
    # Try filling with median for same site, day, and hour
    df[column] = df.groupby(['site_no', 'day_of_year', 'hour_of_day'])[column].transform(lambda x: x.fillna(x.median()))
    
    # If still NaN, try filling with median for same site and day
    df[column] = df.groupby(['site_no', 'day_of_year'])[column].transform(lambda x: x.fillna(x.median()))
    
    # If still NaN, try filling with median for same site
    df[column] = df.groupby(['site_no'])[column].transform(lambda x: x.fillna(x.median()))
    
    # If still NaN, fill with overall median
    remaining_nan_count = df[column].isna().sum()
    if remaining_nan_count > 0:
        print(f"Using overall median to fill {remaining_nan_count} NaN values in column '{column}'")
        df[column] = df[column].fillna(df[column].median())
    
    final_nan_count = df[column].isna().sum()
    print(f"Column '{column}': {original_nan_count} NaNs initially, {final_nan_count} NaNs remaining")

    return df

In [ ]:
df2 = fill_with_median(df, '00060')

In [ ]:
df2.isna().sum()

In [ ]:
df2.describe()

In [ ]:
# Get unique site numbers from the DataFrame
unique_site_numbers = df['site_no'].unique()

# Get the colormap and generate the required number of colors
cmap = plt.colormaps.get_cmap('tab10')  # Get the colormap
colors = [cmap(i / len(unique_site_numbers)) for i in range(len(unique_site_numbers))]  # Generate the colors

# Create a dictionary to map each site number to a color
site_color_map = {site: colors[i] for i, site in enumerate(unique_site_numbers)}

# Scatter plot with colors based on site numbers
plt.scatter(df2['00060'],
            df2['00065'],
            c=df2['site_no'].map(site_color_map),  # Map each site to its corresponding color
            alpha=0.7,
            s=7)

plt.xlabel('Discharge (cfs)')  # Label for x-axis
plt.ylabel('Gauge Height (feet)')  # Label for y-axis
plt.title('USGS Streamflow Data')  # Title for the plot

plt.xscale('log')  # Applying log scale to the x-axis

# Add a legend
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=site, markerfacecolor=color, markersize=5) for site, color in site_color_map.items()]
plt.legend(handles=legend_handles)

plt.grid(True)  # Adding a grid to the plot

plt.show()

In [28]:
df3 = df2.drop(['00065','00065_cd', '00060_cd'], axis=1)

Confirming how many nans now:

In [ ]:
df3.isna().sum()

In [ ]:
original_shape = df3.shape[0]
df3 = df3.drop_duplicates(keep='first')  # Explicitly keep the first occurrence
new_shape = df3.shape[0]
print(f"Removed {original_shape - new_shape} duplicate rows")

In [ ]:
(df3.isna().sum() / len(df3)) * 100

In [ ]:
# Get the list of unique stations
unique_stations_list = df3['site_no'].unique().tolist()

# Initialize an empty DataFrame to store the merged results
merged_df = pd.DataFrame()

# Iterate through each station in the list of unique stations
for station in unique_stations_list:
    # Filter the data for the current station
    station_df = df3[df3['site_no'] == station]
    
    # Check if the station data is not empty
    if not station_df.empty:
        # Rename the '00060' column to include the station's site_no
        new_column_name = f'00060_{station}'
        station_df = station_df.rename(columns={'00060': new_column_name})
        
        # Select only the columns we want to merge
        columns_to_merge = ['year', 'day_of_year', 'hour_of_day', new_column_name]
        station_df = station_df[columns_to_merge]
        
        # If merged_df is empty, this is the first valid data we've processed
        if merged_df.empty:
            merged_df = station_df
        else:
            # If merged_df already contains data, merge the new data
            merged_df = pd.merge(merged_df, station_df, 
                                 on=['year', 'day_of_year', 'hour_of_day'], 
                                 how='outer')
        
        # Print a success message for this station
        print(f"Station {station} processed successfully!")
    else:
        # If station_df is empty, print a message and skip this station
        print(f"Skipping empty result for station {station}")

# After the loop, sort the merged_df by year, day_of_year, and hour_of_day
merged_df = merged_df.sort_values(['year', 'day_of_year', 'hour_of_day'])

# Reset the index of the final merged DataFrame
merged_df = merged_df.reset_index(drop=True)

In [ ]:
merged_df.columns

In [34]:
def fill_with_median_nosite(df, column):
    # Extract site number from the column name
    site_no = column.split('_')[1]
    
    # Store the original NaN count
    original_nan_count = df[column].isna().sum()
    
    # Try filling with median for same day and hour
    df[column] = df.groupby(['day_of_year', 'hour_of_day'])[column].transform(lambda x: x.fillna(x.median()))
    
    # If still NaN, try filling with median for same day
    df[column] = df.groupby(['day_of_year'])[column].transform(lambda x: x.fillna(x.median()))
    
    # If still NaN, fill with overall median
    remaining_nan_count = df[column].isna().sum()
    if remaining_nan_count > 0:
        print(f"Using overall median to fill {remaining_nan_count} NaN values in column '{column}' (site {site_no})")
        df[column] = df[column].fillna(df[column].median())
    
    final_nan_count = df[column].isna().sum()
    print(f"Column '{column}' (site {site_no}): {original_nan_count} NaNs initially, {final_nan_count} NaNs remaining")

    return df

In [ ]:
columns = [
    '00060_06714800',
    '00060_06716100', 
    '00060_06716500', 
    '00060_06718550', 
    '00060_06719505'
]

for column in columns:
    merged_df = fill_with_median_nosite(merged_df, column)

Calculating the missing data per column:

In [ ]:
(merged_df.isna().mean() * 100).round(2)

We have some duplicate rows, we just want 1 answer per row (day of year, year, hour of day)

In [ ]:
# Define the columns to group by and the columns to average
columns_to_group = ['year', 'day_of_year', 'hour_of_day']
columns_to_average = ['00060_06714800', '00060_06716100', '00060_06716500', '00060_06718550', '00060_06719505']

# Group by the specified columns and average the others
result_df = merged_df.groupby(columns_to_group)[columns_to_average].mean().reset_index()

print("Original shape:", merged_df.shape)
print("New shape:", result_df.shape)

# Rename the columns
result_df = result_df.rename(columns={col: f'daily_mean_discharge_{col.split("_")[1]}' for col in columns_to_average})

In [ ]:
result_df.describe()

In [ ]:
# Sample 1000 random rows from result_df
sample_df = result_df.sample(n=1000, random_state=42)

# Select columns to plot (adjust as needed)
columns_to_plot = [col for col in sample_df.columns if col.startswith('daily_mean_discharge')]

# Create the pair plot
sns.pairplot(sample_df[columns_to_plot], kind="scatter", diag_kind="kde", plot_kws={'alpha': 0.3})
plt.tight_layout()
plt.show()

## lets grab some weather station data

In [40]:
url = 'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=0CO&data=p01i&year1=2011&month1=1&day1=1&year2=2023&month2=12&day2=31&tz=America%2FDenver&format=onlycomma&latlon=yes&elev=yes&missing=null&trace=0.0001&direct=no&report_type=3&report_type=4'

In [ ]:
df_met = pd.read_csv(url)
df_met.dtypes

In [42]:
# Convert 'valid' column to datetime format
df_met['valid'] = pd.to_datetime(df_met['valid'])

# Extract year from 'valid' column and create a new 'year' column
df_met['year'] = df_met['valid'].dt.year

# Extract day of year from 'valid' column and create a new 'day_of_year' column
df_met['day_of_year'] = df_met['valid'].dt.dayofyear

# Extract hour from 'valid' column and create a new 'hour_of_day' column
df_met['hour_of_day'] = df_met['valid'].dt.hour

In [ ]:
df_met.describe()

In [ ]:
# Create the first DataFrame with p01i, year, day_of_year, and hour_of_day
df1 = df_met[['p01i', 'year', 'day_of_year', 'hour_of_day']]

print("Shape before dropping NaN:", df1.shape)
df1 = df1.dropna(subset=['p01i'])
print("Shape after dropping NaN:", df1.shape)

# Create the second DataFrame with station, lat, lon, and elevation, and drop duplicates
df_metadata = df_met[['station', 'lat', 'lon', 'elevation']].drop_duplicates()

In [ ]:
# Print the original size
original_size = len(df1)
print(f"Original dataframe size: {original_size} rows")

# 1. Group by year, day_of_year, and hour_of_day, then calculate mean and sum
grouped = df1.groupby(['year', 'day_of_year', 'hour_of_day'])
mean_values = grouped['p01i'].mean().reset_index(name='p01i_mean')
sum_values = grouped['p01i'].sum().reset_index(name='p01i_sum')

# 2. Merge the mean and sum values back to the original dataframe
result = df1.merge(mean_values, on=['year', 'day_of_year', 'hour_of_day'])
result = result.merge(sum_values, on=['year', 'day_of_year', 'hour_of_day'])

# 3. Drop the original 'p01i' column
result = result.drop(columns=['p01i'])

# 4. Remove duplicates based on year, day_of_year, and hour_of_day
result = result.drop_duplicates(subset=['year', 'day_of_year', 'hour_of_day'])

# Print the new size and calculate percentage decrease
new_size = len(result)
print(f"New dataframe size: {new_size} rows")

decrease = original_size - new_size
percentage_decrease = (decrease / original_size) * 100

print(f"Decrease in rows: {decrease}")
print(f"Percentage decrease: {percentage_decrease:.2f}%")

result_met = result

## Merging Datasets

In [46]:
# Assuming your dataframes are named df1 and df2
result = pd.merge(result_met, result_df, on=['year', 'day_of_year', 'hour_of_day'], how='outer')

In [47]:
result = result.dropna(subset=['daily_mean_discharge_06719505']).reset_index(drop=True)

In [ ]:
result.columns

# The Dataframe can be exported here! 

## Splitting up the dataframe

In [ ]:
# Define the range of years
all_years = list(range(2011, 2024))  # 2011 to 2023 inclusive

# Split the years into train, validation, and test
train_years = all_years[:9]  # 2011 to 2019
val_years = all_years[9:11]  # 2020 to 2021
test_years = all_years[11:]  # 2022 to 2023

# Print the year ranges for verification
print("Train years:", train_years)
print("Validation years:", val_years)
print("Test years:", test_years)

# Target variable
target_col = 'daily_mean_discharge_06719505'

# Using 'result' as the dataframe name
# Split the data based on the year lists
X_train = result[result['year'].isin(train_years)].drop(columns=[target_col])
y_train = result[result['year'].isin(train_years)][target_col]

X_val = result[result['year'].isin(val_years)].drop(columns=[target_col])
y_val = result[result['year'].isin(val_years)][target_col]

X_test = result[result['year'].isin(test_years)].drop(columns=[target_col])
y_test = result[result['year'].isin(test_years)][target_col]

# Drop the 'year' column from X_train, X_val, and X_test
X_train = X_train.drop(columns=['year'])
X_val = X_val.drop(columns=['year'])
X_test = X_test.drop(columns=['year'])

# Print the shapes of the resulting datasets
print(f"\nTraining set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Testing set shape: {X_test.shape}")

# Print the column names to verify 'year' has been dropped
print("\nColumns in X_train:", X_train.columns.tolist())

result = result.astype({col: float for col in result.select_dtypes(include=['int32']).columns})


In [ ]:
is_all_float64 = all(result.dtypes == np.float64)

message = (
    "All columns in the DataFrame are dtype float64."
    if is_all_float64
    else "Not all columns in the DataFrame are dtype float64."
)

print(message)

## Machine Learning

In [ ]:
# Create and train the XGBoost model
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42,
    early_stopping_rounds=10
)

# Fit the model with early stopping
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

# Make predictions
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Calculate RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

# Calculate R2
r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)
r2_test = r2_score(y_test, y_test_pred)

# Print results
print("Training Results:")
print(f"RMSE: {rmse_train:.4f}")
print(f"R2: {r2_train:.4f}")

print("\nValidation Results:")
print(f"RMSE: {rmse_val:.4f}")
print(f"R2: {r2_val:.4f}")

print("\nTest Results:")
print(f"RMSE: {rmse_test:.4f}")
print(f"R2: {r2_test:.4f}")

# Feature importance
importance = model.feature_importances_
feature_importance = sorted(zip(importance, X_train.columns), reverse=True)
print("\nTop Features:")
for score, feature in feature_importance:
    print(f"{feature}, {score:.4f}")

In [ ]:
def plot_prediction_scatter(y_true, y_pred, title, ax):
    ax.scatter(y_true, y_pred, alpha=0.2)
    
    # Add 1-to-1 line
    ax_min = min(ax.get_xlim()[0], ax.get_ylim()[0])
    ax_max = max(ax.get_xlim()[1], ax.get_ylim()[1])
    ax.plot([ax_min, ax_max], [ax_min, ax_max], 'r--', lw=2)
    
    ax.set_xlabel('True Values')
    ax.set_ylabel('Predicted Values')
    ax.set_title(title)
    
    # Add R-squared value to the plot
    r2 = r2_score(y_true, y_pred)
    ax.text(0.05, 0.95, f'R² = {r2:.4f}', transform=ax.transAxes, 
            verticalalignment='top')

# Create the figure and subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Plot for training data
plot_prediction_scatter(y_train, y_train_pred, 'Training Set', ax1)

# Plot for validation data
plot_prediction_scatter(y_val, y_val_pred, 'Validation Set', ax2)

# Plot for test data
plot_prediction_scatter(y_test, y_test_pred, 'Test Set', ax3)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
def plot_prediction_hexbin(y_true, y_pred, title, ax):
    hb = ax.hexbin(y_true, y_pred, gridsize=9, cmap='viridis')
    
    # Set vmax to 1/3 of the actual maximum
    vmax = hb.get_array().max() / 3
    hb.set_clim(vmax=vmax)
    
    # Add 1-to-1 line
    ax_min = min(ax.get_xlim()[0], ax.get_ylim()[0])
    ax_max = max(ax.get_xlim()[1], ax.get_ylim()[1])
    ax.plot([ax_min, ax_max], [ax_min, ax_max], 'r--', lw=2)
    
    ax.set_xlabel('True Values')
    ax.set_ylabel('Predicted Values')
    ax.set_title(title)
    
    # Add R-squared value to the plot
    r2 = r2_score(y_true, y_pred)
    ax.text(0.05, 0.95, f'R² = {r2:.4f}', transform=ax.transAxes, 
            verticalalignment='top', color='white')
    
    return hb

# Create the figure and subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Plot for training data
hb1 = plot_prediction_hexbin(y_train, y_train_pred, 'Training Set', ax1)
fig.colorbar(hb1, ax=ax1, label='Count', extend='max')

# Plot for validation data
hb2 = plot_prediction_hexbin(y_val, y_val_pred, 'Validation Set', ax2)
fig.colorbar(hb2, ax=ax2, label='Count', extend='max')

# Plot for test data
hb3 = plot_prediction_hexbin(y_test, y_test_pred, 'Test Set', ax3)
fig.colorbar(hb3, ax=ax3, label='Count', extend='max')

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
num_runs = 50  # Increased number of runs
all_importances = []
all_gain_importances = []
all_cover_importances = []

# Define a range for each hyperparameter
n_estimators_range = range(50, 150)
max_depth_range = range(3, 8)
learning_rate_range = [0.01, 0.05, 0.1, 0.2]

for run in range(num_runs):
    print(f"\nRun {run + 1}/{num_runs}")
    
    # Bootstrap sampling
    X_train_boot, y_train_boot = resample(X_train, y_train, n_samples=len(X_train))
    
    # Randomly select hyperparameters
    n_estimators = np.random.choice(n_estimators_range)
    max_depth = np.random.choice(max_depth_range)
    learning_rate = np.random.choice(learning_rate_range)
    
    # Create and train the XGBoost model
    model = XGBRegressor(
        objective='reg:squarederror',
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        random_state=np.random.randint(1000)  # Random state for each run
    )

    # Fit the model with early stopping
    model.fit(
        X_train_boot, y_train_boot,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # Make predictions
    y_test_pred = model.predict(X_test)

    # Calculate RMSE and R2 for test set
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    print(f"Test RMSE: {rmse_test:.4f}, R2: {r2_test:.4f}")

    # Store different types of feature importances
    all_importances.append(model.feature_importances_)
    all_gain_importances.append(model.get_booster().get_score(importance_type='gain'))
    all_cover_importances.append(model.get_booster().get_score(importance_type='cover'))

# Convert to DataFrames
importance_df = pd.DataFrame(all_importances, columns=X_train.columns)
gain_df = pd.DataFrame(all_gain_importances).fillna(0)
cover_df = pd.DataFrame(all_cover_importances).fillna(0)

# Function to create and show boxplot
def plot_importance(df, title):
    median_importance = df.median().sort_values(ascending=False)
    sorted_features = median_importance.index
    plt.figure(figsize=(12, 8))
    df[sorted_features].boxplot(vert=False, figsize=(12, 8))
    plt.title(title)
    plt.xlabel('Importance Score')
    plt.tight_layout()
    plt.show()

# Plot for each importance type
plot_importance(importance_df, 'Feature Importance Distribution (weight)')
plot_importance(gain_df, 'Feature Importance Distribution (gain)')
plot_importance(cover_df, 'Feature Importance Distribution (cover)')

# Print median feature importance for each type
for df, imp_type in [(importance_df, "Weight"), (gain_df, "Gain"), (cover_df, "Cover")]:
    print(f"\nMedian Feature Importance ({imp_type}):")
    median_importance = df.median().sort_values(ascending=False)
    for feature, importance in median_importance.items():
        print(f"{feature}: {importance:.4f}")